In [62]:
import pandas as pd
import numpy as np

In [63]:
# 上证指数日对数收益率计算
SZZS = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS080101-190131.csv',index_col=0)

SZZS_p = pd.DataFrame(SZZS['close'],index=SZZS.index)

SZZS_p['close-1'] = SZZS_p['close'].shift(1)

SZZS_lnR = pd.DataFrame(np.log(SZZS_p['close'])-np.log(SZZS_p['close-1']),index=SZZS.index,columns=['lnR']).dropna(axis=0)

SZZS_lnR_2 = pd.DataFrame(SZZS_lnR['lnR'].apply(lambda x:x*x))

SZZS_lnR_2.columns = ['lnR_2']

# pd.concat([SZZS_lnR,SZZS_lnR_2],axis=1).to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_r_new.csv')

In [64]:
SZZS_temp = SZZS[['open','close','high','low']]
SZZS_r = pd.concat([SZZS_lnR,SZZS_lnR_2,SZZS_temp,],axis=1,sort=True)

In [65]:
# 上证指数已实现波动率计算
df_fminSZZS = pd.read_csv(r'C:\Users\jxjsj\Desktop\JupyterHome\Data\SZZS_five_min080101-190131.csv',index_col=0,header=0)

df_fminSZZS['time'] = list(df_fminSZZS.index)

df_fminSZZS['date'] = list(df_fminSZZS['time'].apply(lambda x: str(x)[:10]))

df_fminSZZS

df_fminSZZS['close-1'] = df_fminSZZS['close'].shift(1)

df_fminSZZS_lnR_temp = pd.DataFrame(np.log(df_fminSZZS['close'])-np.log(df_fminSZZS['close-1']),index=df_fminSZZS.index,columns=['lnR'])

df_fminSZZS_lnR = pd.concat([df_fminSZZS_lnR_temp,pd.DataFrame(list(df_fminSZZS['date']),columns=['date'],index=df_fminSZZS.index)],axis=1).dropna(axis=0,how='any')

df_fminSZZS_lnR['lnR^2'] = df_fminSZZS_lnR['lnR'].apply(lambda x: x*x)

groupby_temp = df_fminSZZS_lnR['lnR^2'].groupby(df_fminSZZS_lnR['date'])

groupby_df = pd.DataFrame(groupby_temp.sum())

rvSZZS = pd.DataFrame(list(groupby_temp.sum()),columns=['szzsRV'],index=groupby_df.index)

In [66]:
df_temp1 = pd.merge(SZZS_r,rvSZZS, left_index=True,right_index=True)

In [67]:
# 月份哑变量设计
time_temp = pd.date_range('1/1/2008','12/31/2019')

time_df = pd.DataFrame(time_temp,columns=['date'])

month_lst = list(time_df['date'].apply(lambda x: str(x)[5:7]))

month_df = pd.DataFrame(month_lst,columns=['month'])

month_temp = pd.get_dummies(month_df)

time_month_temp = pd.concat([time_df,month_temp],axis=1)

time_month_temp.columns = ['date','Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec']

time_month = time_month_temp[['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec']]

time_month.index=list(time_month_temp['date'])

In [68]:
df_temp2 = pd.merge(df_temp1,time_month, left_index=True,right_index=True)

In [69]:
# 星期哑变量设计
time_temp = pd.date_range('1/1/2008','12/31/2019')

time_df = pd.DataFrame(time_temp,columns=['date'])

lst_week = []
n=627
while n>0:
    for i in [2,3,4,5,6,7,1]:
        lst_week.append(i)
    n-=1

week_temp = pd.get_dummies(pd.DataFrame(lst_week[:4383],columns=['week'])['week'])

time_week_temp = pd.concat([time_df,week_temp],axis=1)

time_week_temp.columns = ['date','M','Tu','W','Th',"F","Sa",'Su']

time_week = time_week_temp[['M','Tu','W','Th',"F","Sa",'Su']]

time_week.index=list(time_week_temp['date'])

In [70]:
df_temp3 = pd.merge(df_temp2,time_week, left_index=True,right_index=True)

In [71]:
# 休市天数计算
import datetime
notrade = pd.DataFrame(list(SZZS.index),columns=['date'])

notrade['date-1'] = notrade['date'].shift(1)

notrade = notrade.dropna(axis=0,how='any')

notrade_daycnt_temp = pd.DataFrame(notrade['date']+notrade['date-1'],columns=['daycnt'])

notrade_daycnt_lst = list(
    notrade_daycnt_temp['daycnt'].apply(
    lambda x: (datetime.datetime.strptime(x[:10], "%Y-%m-%d")-datetime.datetime.strptime(x[10:20], "%Y-%m-%d")).days-1
                         ))

notrade_daycnt = pd.DataFrame(notrade_daycnt_lst,columns=['ntd_cnt'])
notrade_daycnt.index = list(notrade['date'])

In [72]:
df_temp4 = pd.merge(df_temp3,notrade_daycnt, left_index=True,right_index=True)

In [73]:
# 换手率交易量
SZZS_motion = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_motion090101-190227.csv',index_col=0)

SZZS_motion['volume-1'] = SZZS_motion['volume'].shift(1)
SZZS_motion['volumeR'] = np.log(SZZS_motion['volume']) - np.log(SZZS_motion['volume-1'])

SZZS_motion['money-1'] = SZZS_motion['money'].shift(1)
SZZS_motion['moneyR'] = np.log(SZZS_motion['money']) - np.log(SZZS_motion['money-1'])

SZZS_motion['turnoverR'] = SZZS_motion['turnover_ratio']/100

SZZS_motion_temp = SZZS_motion[['volume','money','volumeR','moneyR','turnoverR']]

In [74]:
df_temp5 = pd.merge(df_temp4,SZZS_motion_temp,how='left' ,left_index=True,right_index=True)
df_temp5 = df_temp5.fillna(0)

In [75]:
# 前日新闻条数
SZZS_new = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news_cnt_090101-190201.csv',index_col=0)

df_temp6 = pd.merge(df_temp5,SZZS_new,how='left' ,left_index=True,right_index=True)

In [80]:
# 5日均收益率
df_temp6['lnRroll5'] = df_temp6['lnR'].rolling(5).mean()

# 20日均换手率
df_temp6['turnoverRroll20'] = df_temp6['turnoverR'].rolling(20).mean()

# 20日均收益率
df_temp6['lnRroll20'] = df_temp6['lnR'].rolling(20).mean()

# 20日标准差
df_temp6['stdlnRroll20'] = df_temp6['lnR'].rolling(20).std()

# 60日方差
df_temp6['varlnRroll60'] = df_temp6['lnR'].rolling(20).var()

######################################################################################
# 60日均收益率
df_temp6['lnRroll60'] = df_temp6['lnR'].rolling(60).mean()

# 60日标准差
df_temp6['stdlnRroll60'] = df_temp6['lnR'].rolling(60).std()

# 60日夏普 无风险0.04
df_temp6['sharpe60'] = (df_temp6['lnRroll60']*250 - 0.04) / (df_temp6['stdlnRroll60']*(250**0.5))

# 60日夏普倒数
df_temp6['sharpe60'] = df_temp6['sharpe60'].apply(lambda x: 1/x)
######################################################################################

# 威廉变异离散量 (收盘价－开盘价)/(最高价－最低价)×成交量，再做加和，使用过去6个交易日的数据
df_temp6['WVAD_temp'] = (df_temp6.close-df_temp6.open)/(df_temp6.high-df_temp6.low)*df_temp6.volume
df_temp6['WVAD'] = df_temp6['WVAD_temp'].rolling(6).sum()

# 60日负偏度
df_temp6['skewlnR60'] = -df_temp6['lnR'].rolling(60).skew()

# 60日峰度
df_temp6['kurtlnR60'] = df_temp6['lnR'].rolling(60).kurt()

df_temp6 = df_temp6.fillna(0)

In [82]:
# 主成分分析    
from sklearn.decomposition import PCA

In [95]:
# 短期风险因素

# 合成成分
risk_pd = pd.concat([df_temp6['skewlnR60'],df_temp6['sharpe60'],df_temp6['kurtlnR60'],df_temp6['varlnRroll60']],axis=1)

# 标准化 - 统一量级、分布 (有问题！df前400行很多0！！)
for i in risk_pd.columns:
    risk_pd[i] = risk_pd[i].apply(lambda x: (x-risk_pd[i].mean())/risk_pd[i].std())

pca=PCA(n_components=1)
newData=pca.fit_transform(risk_pd)
print(pca.explained_variance_ratio_)

df_temp6['risk'] = newData

[0.36264464]


In [94]:
# 投资者情绪因素

# 合成成分
emotion_pd = pd.concat([df_temp6['turnoverRroll20'],df_temp6['WVAD']],axis=1)

# 标准化 - 统一量级、分布 (有问题！df前400行很多0！！)
for i in emotion_pd.columns:
    emotion_pd[i] = emotion_pd[i].apply(lambda x: (x-emotion_pd[i].mean())/emotion_pd[i].std())

pca=PCA(n_components=1)
newData=pca.fit_transform(emotion_pd)
print(pca.explained_variance_ratio_)

df_temp6['emotion'] = newData

[0.61886875]


In [85]:
# 政策因素

df_temp6['news'] = df_temp6['content']

In [86]:
var_lst = ['risk','emotion','news']
col_lst = ['M', 'Tu', 'Th', 'F','Jan','Feb','May','Dec','ntd_cnt']
for var in var_lst:
    for column in col_lst:
        column_name = var + '_' + column
        df_temp6[column_name] = df_temp6[var]*df_temp6[column]

In [87]:
df_temp7 = df_temp6[['lnR', 'lnR_2', 'szzsRV', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'June',
       'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'M', 'Tu', 'W', 'Th', 'F',
       'Sa', 'Su', 'ntd_cnt', 'risk_M', 'risk_Tu', 'risk_Th', 'risk_F', 'risk_Jan',
       'risk_Feb', 'risk_May', 'risk_Dec', 'risk_ntd_cnt', 'emotion_M',
       'emotion_Tu', 'emotion_Th', 'emotion_F', 'emotion_Jan', 'emotion_Feb',
       'emotion_May', 'emotion_Dec', 'emotion_ntd_cnt', 'news_M', 'news_Tu',
       'news_Th', 'news_F', 'news_Jan', 'news_Feb', 'news_May', 'news_Dec',
       'news_ntd_cnt']]

In [88]:
df_temp7.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_lnr_rv_w_m_ntd_080101-190131adj.csv')

In [89]:
list(df_temp7.columns).index('risk_ntd_cnt')+1

32